In [1]:
!pip install python-docx PyPDF2 pytesseract

In [2]:
pip install SpeechRecognition

In [3]:
!pip install gTTS

In [4]:
import os
from docx import Document
import openpyxl
import PyPDF2
from PIL import Image
import pytesseract
from io import BytesIO

# --- Updated extraction functions ---

def extract_text_from_docx(file_obj):
    doc = Document(file_obj)
    return "\n".join([para.text for para in doc.paragraphs])


def extract_text_from_excel(file_obj):
    wb = openpyxl.load_workbook(file_obj, data_only=True)
    text = ""
    for sheet in wb:
        for row in sheet.iter_rows(values_only=True):
            row_text = [str(cell) if cell is not None else "" for cell in row]
            text += "\t".join(row_text) + "\n"
    return text


def extract_text_from_pdf(file_obj):
    text = ""
    reader = PyPDF2.PdfReader(file_obj)
    for page in reader.pages:
        text += page.extract_text() or ''
    return text


def extract_text_from_image(file_obj):
    image = Image.open(file_obj)
    text = pytesseract.image_to_string(image)
    return text


# --- Main wrapper function ---

def report_to_text(file_obj, filename):
    ext = os.path.splitext(filename)[-1].lower()

    if ext == ".docx":
        return extract_text_from_docx(file_obj)
    elif ext in [".xlsx", ".xls"]:
        return extract_text_from_excel(file_obj)
    elif ext == ".pdf":
        return extract_text_from_pdf(file_obj)
    elif ext in [".png", ".jpg", ".jpeg"]:
        return extract_text_from_image(file_obj)
    else:
        return "Unsupported file format."


# report_to_text(file_obj, filename)
# --- Example usage for Colab or web upload ---

# if __name__ == "__main__":
#     from google.colab import files  # Remove this if running outside Colab

#     uploaded = files.upload()  # Upload one file
#     for filename, file_data in uploaded.items():
#         with BytesIO(file_data) as file_obj:
#             text = report_to_text(file_obj, filename)
#             print("Extracted Text:\n", text)


In [5]:
# apt-get install git

# Clone the repository
# git clone https://huggingface.co/deepset/xlm-roberta-large-squad2

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch



use_local_model = False
# If using a remote model (e.g., on the Hugging Face Hub), specify the exact model ID
REMOTE_MODEL_ID = "deepset/roberta-large-squad2"

# Chunking and Summarization
def summarize_text_across_chunks(text, summarizer, chunk_size=700, overlap=100):
    tokens = text.split()
    summaries = []
    start = 0

    while start < len(tokens):
        end = min(start + chunk_size, len(tokens))
        chunk = " ".join(tokens[start:end])
        try:
            summary = summarizer(chunk, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
            summaries.append(summary.strip())
        except Exception as e:
            print(f"[Warning] Summarization failed on chunk: {e}")
        start += chunk_size - overlap

    # Optionally join all chunk summaries into a final summary
    if len(summaries) == 1:
        return summaries[0]
    else:
        final_summary = summarizer(" ".join(summaries), max_length=100, min_length=50, do_sample=False)[0]['summary_text']
        return final_summary

# Load Summarization Pipeline
def get_summarizer(use_local=False, model_id="facebook/bart-large-cnn"):
    """
    Loads a summarization model pipeline.
    """
    if use_local:
        print("[INFO] Local model loading is not supported here.")
        return None

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda" if torch.cuda.is_available() else "cpu")

    summarizer = pipeline(
        "summarization",
        model=model,
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1
    )
    return summarizer

# Extract Summary from Text
def extract_text_summary(text, summarizer):
    return summarize_text_across_chunks(text, summarizer)

def intialize_rag_model():
  ragModel = get_summarizer(use_local_model)
  return ragModel

text = "Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great,Hello world, this world is beautiful and amzaing , my friends are great"
# extract_text_summary(text, ragModel)

ragModel = get_summarizer(use_local_model)
extract_text_summary(text, ragModel)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


'Hello world, this world is beautiful and amzaing , my friends are great. Hello world, \xa0this world is\xa0beautiful\xa0and\xa0amzaing \xa0my\xa0friends are great,Hello world\xa0this\xa0world is beautiful\xa0and amzaed , my\xa0friends\xa0are great.'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 13.1 MB/s eta 0:00:00


In [8]:
import gradio as gr
import speech_recognition as sr
import os
# from a import report_to_text
# from robertaModel import extract_text_summary, intialize_rag_model

# Placeholder for document summarization
def summarize_document(file, file_name):
    text = report_to_text(file, file_name)
    return extract_text_summary(text, ragModel)
    # return text

# Placeholder for processing voice input
def process_voice_input(transcribed_text):
    return f"Processed response to: {transcribed_text}"

# Handle file upload and enable voice chat
def handle_file_upload(file):
    file_name = os.path.basename(file.name) if hasattr(file, "name") else str(file)
    print("Uploaded file name:", file_name)
    summary = summarize_document(file, file_name)
    return summary, gr.update(interactive=True)

# Handle voice input and convert to text
def handle_audio_input(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            print(f"Recognized text: {text}")
            response = process_voice_input(text)
            return response
        except sr.UnknownValueError:
            return "Sorry, could not understand the audio."
        except sr.RequestError:
            return "Speech recognition service is unavailable."

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 📄 Voice Reports
        **Upload a document to get a summary and enable voice chat!**
        <br>
        <span style='color:gray'>Step 1: Upload your document.<br>
        Step 2: Review the summary.<br>
        Step 3: Use your microphone to ask questions about the document.</span>
        """,
        elem_id="main-title"
    )

    with gr.Accordion("📤 Document Upload & Summary", open=True):
        with gr.Row():
            file_input = gr.File(label="Upload Document")
            summary_output = gr.Textbox(label="Summary", lines=10, interactive=False, show_copy_button=True)

    with gr.Accordion("🎤 Voice Chat", open=True):
        with gr.Row():
            mic_input = gr.Audio(sources=["microphone"], type="filepath", label="Speak Now", interactive=False)
            chat_output = gr.Textbox(label="Response", lines=4, interactive=False, show_copy_button=True)

    file_input.upload(fn=handle_file_upload, inputs=file_input, outputs=[summary_output, mic_input])
    mic_input.change(fn=handle_audio_input, inputs=mic_input, outputs=chat_output)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://562ce5a4855d3d8df6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
import gradio as gr
import speech_recognition as sr
import os
from gtts import gTTS
import tempfile
import os

text_file = ""

def summarize_document(file):
    file_name = os.path.basename(file.name) if hasattr(file, "name") else str(file)
    text_file = report_to_text(file, file_name)
    return extract_text_summary(text_file, ragModel)

def summary_text_to_speech(summary_text):
    if not summary_text or summary_text.strip() == "":
        return None
    tts = gTTS(text=summary_text, lang='en')
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts.save(fp.name)
        print(fp.name)
        return fp.name

# def summary_text_to_speech(summary_text):
#     if not summary_text or summary_text.strip() == "":
#         return None
#     tts = gTTS(text=summary_text, lang='en')
#     audio_path = "summary_tts.mp3"
#     tts.save(audio_path)
#     return "/content/summary_tts.mp3"

def handle_file_upload(file, file_count):
    file_name = os.path.basename(file.name) if hasattr(file, "name") else str(file)
    print("Uploaded file name:", file_name)
    summary = summarize_document(file)
    file_count += 1  # Increment count
    summary_audio = summary_text_to_speech(summary)
    # Enable voice chat after upload
    return (
        summary,
        gr.update(value=None, interactive=True),
        f"Files uploaded this session: {file_count}",
        file_count,
        summary_audio,
        gr.update(interactive=True),  # Enable mic_input
    )

def process_voice_input(transcribed_text):
    # Simulate sending the user's question to the AI backend and getting a response.
    # Replace this with your actual AI backend call.
    # ai_response = f"{transcribed_text}"
    # return ai_response
    # transcribed_text
    return extract_text_summary( text_file, ragModel)

def tts_response(text):
    tts = gTTS(text=text, lang='en')
    audio_path = "ai_response.mp3"
    tts.save(audio_path)
    return audio_path

def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            user_text = recognizer.recognize_google(audio_data, language="en-US")
            return user_text
        except sr.UnknownValueError:
            return "[Unrecognized speech]"
        except sr.RequestError:
            return "[Speech recognition service unavailable]"

def handle_voice_chat(audio_file, chat_history):
    if audio_file is None:
        return chat_history, None, "\n".join([f"You: {u}\nAI: {a}" for u, a in zip(chat_history[::2], chat_history[1::2])]) if chat_history else ""
    user_text = transcribe_audio(audio_file)
    chat_history = chat_history or []
    chat_history.append(f"You: {user_text}")
    ai_text = process_voice_input(user_text)
    chat_history.append(f"AI: {ai_text}")
    ai_audio = tts_response(ai_text)
    conversation_display = "\n\n".join(chat_history)
    return chat_history, ai_audio, conversation_display

def export_conversation(chat_history):
    if not chat_history:
        return None
    conversation_text = "\n\n".join(chat_history)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".txt", mode="w", encoding="utf-8") as f:
        f.write(conversation_text)
        return f.name

# --- Gradio UI ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 📄 Voice Reports
        **Upload a document to get a summary and enable voice chat!**
        """
    )

    # --- File Upload & Summary Block ---
    with gr.Group():
        gr.Markdown("## 📤 Document Upload & Summary")
        file_count_state = gr.State(0)
        with gr.Row():
            file_input = gr.File(
                label="Upload Document",
                interactive=True
            )
            summary_output = gr.Textbox(label="Summary", lines=10, interactive=False, show_copy_button=True)
        file_count_display = gr.Markdown("Files uploaded this session: 0")
        gr.Markdown("**Tip:** Uploading a new file will automatically replace the previous one.")
        with gr.Row():
            tts_audio = gr.Audio(label="Summary Audio", interactive=True, autoplay=True)

        file_input.upload(
            fn=handle_file_upload,
            inputs=[file_input, file_count_state],
            outputs=[
                summary_output,
                file_input,
                file_count_display,
                file_count_state,
                tts_audio,
                # mic_input will be enabled after upload
                # send_btn removed
            ]
        )

    # --- Voice Chat Block ---
    with gr.Group():
        gr.Markdown("## 🎤 Voice Chat")
        conversation_state = gr.State([])
        with gr.Row():
            mic_input = gr.Audio(
                sources=["microphone"],
                type="filepath",
                label="Speak Now",
                interactive=False  # Disabled until file upload
            )
            ai_audio = gr.Audio(label="AI Speaking", interactive=False, autoplay=True)
        chat_output = gr.Textbox(label="Conversation", lines=12, interactive=False, show_copy_button=True)
        with gr.Row():
            export_btn = gr.Button("Export Conversation")
            export_file = gr.File(label="Download Conversation")

        # Automatically process voice when recording stops
        mic_input.change(
            fn=handle_voice_chat,
            inputs=[mic_input, conversation_state],
            outputs=[conversation_state, ai_audio, chat_output]
        )

        export_btn.click(
            fn=export_conversation,
            inputs=[conversation_state],
            outputs=[export_file]
        )

    # Enable mic_input after file upload
    file_input.upload(
        fn=handle_file_upload,
        inputs=[file_input, file_count_state],
        outputs=[
            summary_output,
            file_input,
            file_count_display,
            file_count_state,
            tts_audio,
            mic_input  # Enable mic_input after upload
        ]
    )

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2fe74ffb6f182ada0a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [17]:
import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import uuid
import os
import edge_tts
import asyncio
import io

# ------------------------------
# Placeholder summarization logic
# ------------------------------
def summarize_document(file):
    return "This is a placeholder summary. Replace this with actual summary logic."

# ------------------------------
# Save TTS to /content for Colab compatibility
# ------------------------------
async def text_to_speech_bytes(text):
    communicate = edge_tts.Communicate(text, "en-US-AriaNeural")
    stream = io.BytesIO()
    async for chunk in communicate.stream():
        if chunk["type"] == "audio":
            stream.write(chunk["data"])
    stream.seek(0)
    return stream.read()

def tts_response(text):
    tts = gTTS(text=text, lang='en')
    file_path = f"/content/ai_response_{uuid.uuid4().hex}.mp3"
    tts.save(file_path)
    return file_path

# ------------------------------
# File upload handling
# ------------------------------
def handle_file_upload(file, file_count):
    file_name = os.path.basename(file.name) if hasattr(file, "name") else str(file)
    print("Uploaded file name:", file_name)
    summary = summarize_document(file)
    file_count += 1
    summary_audio = summary_text_to_speech(summary)
    return (
        summary,
        gr.update(value=None, interactive=True),
        f"Files uploaded this session: {file_count}",
        file_count,
        summary_audio,
        gr.update(interactive=True)
    )

# ------------------------------
# Voice chat processing
# ------------------------------
def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            user_text = recognizer.recognize_google(audio_data, language="en-US")
            return user_text
        except sr.UnknownValueError:
            return "[Unrecognized speech]"
        except sr.RequestError:
            return "[Speech recognition service unavailable]"

def process_voice_input(transcribed_text):
    # Simulated AI response
    return f"{transcribed_text}"

def handle_voice_chat(audio_file, chat_history):
    if audio_file is None:
        return chat_history, None, "\n\n".join(chat_history) if chat_history else ""
    user_text = transcribe_audio(audio_file)
    chat_history = chat_history or []
    chat_history.append(f"You: {user_text}")
    ai_text = process_voice_input(user_text)
    chat_history.append(f"AI: {ai_text}")
    ai_audio = tts_response(ai_text)
    conversation_display = "\n\n".join(chat_history)
    return chat_history, ai_audio, conversation_display

# ------------------------------
# Export chat to /content for download
# ------------------------------
def export_conversation(chat_history):
    if not chat_history:
        return None
    conversation_text = "\n\n".join(chat_history)
    file_path = f"/content/conversation_{uuid.uuid4().hex}.txt"
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(conversation_text)
    return file_path

# ------------------------------
# Gradio UI
# ------------------------------
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 📄 Voice Reports
        **Upload a document to get a summary and enable voice chat!**
        """
    )

    # File upload section
    with gr.Group():
        gr.Markdown("## 📤 Document Upload & Summary")
        file_count_state = gr.State(0)
        with gr.Row():
            file_input = gr.File(label="Upload Document", interactive=True)
            summary_output = gr.Textbox(label="Summary", lines=10, interactive=False, show_copy_button=True)
        file_count_display = gr.Markdown("Files uploaded this session: 0")
        gr.Markdown("**Tip:** Uploading a new file will automatically replace the previous one.")
        with gr.Row():
            tts_audio = gr.Audio(label="Summary Audio", interactive=False, autoplay=True)

        file_input.upload(
            fn=handle_file_upload,
            inputs=[file_input, file_count_state],
            outputs=[
                summary_output,
                file_input,
                file_count_display,
                file_count_state,
                tts_audio,
                # mic_input will be enabled after upload
            ]
        )

    # Voice chat section
    with gr.Group():
        gr.Markdown("## 🎤 Voice Chat")
        conversation_state = gr.State([])
        with gr.Row():
            mic_input = gr.Audio(sources=["microphone"], type="filepath", label="Speak Now", interactive=False)
            ai_audio = gr.Audio(label="AI Speaking", interactive=False, autoplay=True)
        chat_output = gr.Textbox(label="Conversation", lines=12, interactive=False, show_copy_button=True)
        with gr.Row():
            export_btn = gr.Button("Export Conversation")
            export_file = gr.File(label="Download Conversation")

        mic_input.change(
            fn=handle_voice_chat,
            inputs=[mic_input, conversation_state],
            outputs=[conversation_state, ai_audio, chat_output]
        )

        export_btn.click(
            fn=export_conversation,
            inputs=[conversation_state],
            outputs=[export_file]
        )

    # Enable mic input after upload
    file_input.upload(
        fn=handle_file_upload,
        inputs=[file_input, file_count_state],
        outputs=[
            summary_output,
            file_input,
            file_count_display,
            file_count_state,
            tts_audio,
            mic_input
        ]
    )

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://372624cb694d3925ee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
!pip install edge-tts


  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=b856ece9aaf3c7ecdada4eb265dcc2c93c0d1c065d5f92476a489ac810415713
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt


In [19]:
import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import uuid
import os

# --- Placeholder summarization logic ---
def summarize_document(file):
    return "This is a placeholder summary. Replace this with actual summary logic."

# --- Save TTS output to /content for Colab compatibility ---
def summary_text_to_speech(summary_text):
    if not summary_text or summary_text.strip() == "":
        return None
    tts = gTTS(text=summary_text, lang='en')
    file_path = f"/content/summary_{uuid.uuid4().hex}.mp3"
    tts.save(file_path)
    return file_path

def tts_response(text):
    tts = gTTS(text=text, lang='en')
    file_path = f"/content/ai_response_{uuid.uuid4().hex}.mp3"
    tts.save(file_path)
    return file_path

# --- File upload handling ---
def handle_file_upload(file, file_count):
    file_name = os.path.basename(file.name) if hasattr(file, "name") else str(file)
    print("Uploaded file name:", file_name)
    summary = summarize_document(file)
    file_count += 1
    summary_audio = summary_text_to_speech(summary)
    return (
        summary,
        gr.update(value=None, interactive=True),
        f"Files uploaded this session: {file_count}",
        file_count,
        summary_audio,
        gr.update(interactive=True)  # Enable mic input
    )

# --- Voice chat processing ---
def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            user_text = recognizer.recognize_google(audio_data, language="en-US")
            return user_text
        except sr.UnknownValueError:
            return "[Unrecognized speech]"
        except sr.RequestError:
            return "[Speech recognition service unavailable]"

def process_voice_input(transcribed_text):
    return f"{transcribed_text}"

def handle_voice_chat(audio_file, chat_history):
    if audio_file is None:
        return chat_history, None, "\n\n".join(chat_history) if chat_history else ""
    user_text = transcribe_audio(audio_file)
    chat_history = chat_history or []
    chat_history.append(f"You: {user_text}")
    ai_text = process_voice_input(user_text)
    chat_history.append(f"AI: {ai_text}")
    ai_audio = tts_response(ai_text)
    conversation_display = "\n\n".join(chat_history)
    return chat_history, ai_audio, conversation_display

# --- Export conversation to downloadable file ---
def export_conversation(chat_history):
    if not chat_history:
        return None
    conversation_text = "\n\n".join(chat_history)
    file_path = f"/content/conversation_{uuid.uuid4().hex}.txt"
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(conversation_text)
    return file_path

# --- Gradio UI ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 📄 Voice Reports
        **Upload a document to get a summary and enable voice chat!**
        """
    )

    # File Upload & Summary
    with gr.Group():
        gr.Markdown("## 📤 Document Upload & Summary")
        file_count_state = gr.State(0)
        with gr.Row():
            file_input = gr.File(label="Upload Document", interactive=True)
            summary_output = gr.Textbox(label="Summary", lines=10, interactive=False, show_copy_button=True)
        file_count_display = gr.Markdown("Files uploaded this session: 0")
        gr.Markdown("**Tip:** Uploading a new file will automatically replace the previous one.")
        with gr.Row():
            tts_audio = gr.Audio(label="Summary Audio", interactive=False, autoplay=True)

        file_input.upload(
            fn=handle_file_upload,
            inputs=[file_input, file_count_state],
            outputs=[
                summary_output,
                file_input,
                file_count_display,
                file_count_state,
                tts_audio,
                # mic_input enabled later
            ]
        )

    # Voice Chat
    with gr.Group():
        gr.Markdown("## 🎤 Voice Chat")
        conversation_state = gr.State([])
        with gr.Row():
            mic_input = gr.Audio(sources=["microphone"], type="filepath", label="Speak Now", interactive=False)
            ai_audio = gr.Audio(label="AI Speaking", interactive=False, autoplay=True)
        chat_output = gr.Textbox(label="Conversation", lines=12, interactive=False, show_copy_button=True)
        with gr.Row():
            export_btn = gr.Button("Export Conversation")
            export_file = gr.File(label="Download Conversation")

        mic_input.change(
            fn=handle_voice_chat,
            inputs=[mic_input, conversation_state],
            outputs=[conversation_state, ai_audio, chat_output]
        )

        export_btn.click(
            fn=export_conversation,
            inputs=[conversation_state],
            outputs=[export_file]
        )

    # Enable mic after file upload
    file_input.upload(
        fn=handle_file_upload,
        inputs=[file_input, file_count_state],
        outputs=[
            summary_output,
            file_input,
            file_count_display,
            file_count_state,
            tts_audio,
            mic_input
        ]
    )

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://92367b52eb263532e6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
